首先恭喜各位，通过第一轮的简历筛选，进入线上测试环节。

通过这次线上测试，希望大家能够在限定的时间内，尽自己最大努力，完成一些数据处理和机器学习建模任务，展现自己对机器学习建模工作的熟悉程度。

## 任务说明
请在这个jupyter notebook中
* 补充完成任务所需要的python代码。可以直接在code cell中编写代码；或者将代码写在.py文件中后在这个notebook中import使用
* 利用markdown cell回答相关的问题或者解释分析思路

完成后，在自己的github上建立一个repo。将这个notebook，相关的.py文件，保存处理结果的csv文件等，push到建立的repo。将repo链接提交给面试官。


## 注意事项：
1. 使用python3.6以上版本，如果使用```pandas```等python库，请先升级到最新版本。
2. 请确保面试官在clone各位发布的repo后，能够跑通这个notebook中的代码。
3. 除了实现任务所需的功能，编写的代码具有高可维护性，也是我们非常希望候选人具备的能力。请使用可读性强的命名，避免代码冗余，以及体现其他良好的编程实践。（可参考Refactoring by Kent Beck and Martin Fowler一书中关于"bad code smell"的讨论）
4. 这个测试的目的是检验候选人是否具备高效的完成建模相关工作的能力，对相关工具的熟悉程度也是考核的目标之一。不鼓励重复造轮子。建议充分利用```pandas```, ```sklearn```等已有python库完成任务。
5. 请展现自己真实的能力，不要请人代劳。如果候选人试用期阶段表现的实际工作能力明显低于线上测试表现的能力，候选人会被认为采用非诚信手段通过线上测试，将予以坚决辞退。
6. 职场中，Deadline是非常严肃的。无论完成度如何，请务必在截止时间前，提交自己的结果。逾期提交将不予受理。

## 机器学习建模（任务1～任务2）
以下任务基于虚拟的建模数据dataset.csv

建模数据集构成
* id：样本的编号
* dataset：划分了哪些数据用于train, 那些用于test
* label:  0或者1, 只有train数据才给出了label
* 其实字母f开头的列为建模特征

In [1]:
import pandas as pd
model_data = pd.read_csv('model_data.csv')

### 任务1

请根据train数据，使用3种指定的机器学习算法分别训练模型。这3种算法是
* xgboost
* logistic regression
* random forest

要求:
* 预测目标为label列
* 模型优化的目标为auc_roc
* 需要对特征进行筛选。每种算法训练的模型，最后使用的特征控制在20个以内。
* 每种算法得到的模型，将模型在test数据集的预测结果(0~1之间的一个分值)保存在prediciton.csv文件中。在候选人提交结果后，面试官会计算候选人的模型在test数据集的auc_roc。 csv应该包含id, 以及每个模型对应的proba。请参考作为样例的prediciton_sample.csv中的形式

### 任务2

如果需要在产品中使用。你会推荐使用之前哪个算法得到的模型？解释为什么。

答：可能会使用3个模型的融合，每个模型有一个投票权，少数服从多数。实际应用哪个模型会视实际场景而定，Xgboost模型通过调参可以获得很高的准确率，但是相应的模型调参和模型训练会花更多的时间，后期上线会对资源有很大的消耗，LR模型参数少资源消耗小，可解释性好，特征工程做的好准确率尚可，适合快速开发和上线应用。

### 任务3

任务1～任务4体现了从得到建模数据，到完成建模实验的环节。

然而，对于整个建模项目来说，以上环节只是一部分工作内容。请候选人根据自己的经验和理解，描述从接到业务部门的需求开始（假设是一个0，1分类的监督式机器学习任务），到监控模型在产品中的使用情况为止，有哪些重要的工作环节？在这些过程中，机器学习建模工程师和业务部门成员如何分工？如何合作？

建议通过流程图配合必要的文字描述，表达自己的理解。





## 数据处理

### 任务4

history_df中存放了每个id，在不同time，对应的A, B的数值。

In [ ]:
history_data = pd.read_csv('history_data.csv', parse_dates=['time'])
history_data

sample_df存放了一些id和obs_time的组合。

In [ ]:
sample_df = pd.read_csv('sample.csv')
sample_df

对sample_df中每个id和obs_time的组合，计算离obs_time过去10, 60天内， 该id对应的A, B的最大值, 非空记录数量。将结果存在不同的列中。

例如:


最终的结果包含的列:
* id
* obs_time
* 10天内A平均值
* 10天内A非空样本数量
* 60天内A平均值
* 60天内A非空样本数量
* 10天内B平均值
* 10天内B非空样本数量
* 60天内B平均值
* 60天内B非空样本数量

将计算结果存为excel保存

In [ ]:
# 开发相应的代码
#获取过去10, 60天内，该id对应的A,B的最大值,非空记录数量
def Get_A_B_Value(history_data,sample_df):
    lst = []
    for i in range(len(sample_df.index)):
        lst_tmp = []
        lst_tmp.append(sample_df['id'][i])
        lst_tmp.append(sample_df['obs_time'][i])
        
        #计算时间间隔
        history_data_1 = pd.DataFrame()
        history_data_1 = history_data[history_data.id == sample_df['id'][i]]
        history_data_1['time_interval'] = [(pd.to_datetime(sample_df['obs_time'][i]) - j).days for j in history_data_1['time']]
        
        #计算60天内A，B的最大值和空值
        tmp1 = history_data_1.loc[history_data_1.time_interval < 10]
        tmp2 = tmp1[tmp1.time_interval > 0]
        lst_tmp.append(tmp2['A'].mean())
        lst_tmp.append(len(tmp2['A']) - len(tmp2[tmp2['A'].isnull()]))
        lst_tmp.append(tmp2['B'].mean())
        lst_tmp.append(len(tmp2['B']) - len(tmp2[tmp2['B'].isnull()]))
        
        #计算60天内A，B的最大值和空值
        tmp3 = history_data_1.loc[history_data_1.time_interval < 60]
        tmp4 = tmp3[tmp3.time_interval > 0]
        lst_tmp.append(tmp4['A'].mean())
        lst_tmp.append(len(tmp4['A']) - len(tmp4[tmp4['A'].isnull()]))
        lst_tmp.append(tmp4['B'].mean())
        lst_tmp.append(len(tmp4['B']) - len(tmp4[tmp4['B'].isnull()]))
        lst.append(lst_tmp)
    return lst

In [ ]:
result = pd.DataFrame(Get_A_B_Value(history_data,sample_df))
result.columns = ['id','obs_time','10天内A平均值','10天内A非空样本数量','10天内B平均值','10天内B非空样本数量',
                  '60天内A平均值','60天内A非空样本数量','60天内B平均值','60天内B非空样本数量']
result